# Evaluate reconstruction errors in SVD

In [ ]:
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator
import scipy.signal as sig
from tqdm import tqdm
import sys
sys.path.append('../../../utils')
import gc
from TurboFJLT import *
from InstantFrequency import inst_freq
from TurboFJLT_helpers import TurboHDF5Reader

In [ ]:
%config InlineBackend.figure_format='retina'
plt.style.use("../../../mplstyles/paper_half.mplstyle")
cmap_b = mpl.colormaps['Blues']
cmap_r = mpl.colormaps['Reds']
blue = cmap_b(0.75)
red = 'r'
green = 'g'

# Synchronise this with svd.ipynb
e = 0.01
sv_to_show = 20
num_modes_to_visualise = 7
num_snapshots = 500
snapshot_sequence = list(range(num_snapshots))

In [ ]:
Q_file = "../../../data/fine_airfoil_cascade.h5"
dir_svd_file = "../data/direct_svd.h5"
fjlt_svd_file = "../data/fjlt_svd_{}_linking_snapshots.h5"

In [ ]:
def formQ(reader, seq_to_extract):
    q_mf = reader.load_meanflow()
    Q = reader.load_full(seq_to_extract)
    for i in range(Q.shape[1]):
        Q[:, i] -= q_mf
    return Q

In [ ]:
# Print memory usage of each snapshot
reader_temp = TurboHDF5Reader(Q_file)
Q_temp = formQ(reader_temp, [0])
mem_per_snap = Q_temp[:, 0].nbytes
print("Memory per snapshot: {}B".format(mem_per_snap))

In [ ]:
with h5.File(dir_svd_file, 'r') as f:
    dir_s = f["/s"][()]

### Plot the singular values for varying numbers of linking snapshots
Singular value error analysis for various FJLT subspace dimensions (i.e. number of linking snapshots)

In [ ]:
def plot_singular_values(axs, s, s_f):
    num_sing_vals = min([len(s),len(s_f)])
    s, s_f = s[:num_sing_vals], s_f[:num_sing_vals]

    axs[0].plot(np.arange(len(s)), s, color=blue, marker="o", label="Direct")
    axs[0].plot(np.arange(len(s_f)), s_f, color=red, marker='x', label="FJLT")
    axs[1].plot(np.arange(len(s)), 100*np.abs(s-s_f)/s, color=green, marker="*", label="Percentage error")
    axs[1].plot([0, len(s)], [100*e, 100*e], color="k", alpha=0.5, ls="--", label="FJLT distortion thresh.")

    axs[0].set_ylabel(r"$\sigma_i$")
    axs[1].set_ylabel(r"$\%$ error")
    axs[1].set_xlabel(r"Mode index")
    for i in range(2):
        axs[i].legend();
        axs[i].set_xlim([0, sv_to_show])
        axs[i].xaxis.set_major_locator(MaxNLocator(integer=True))

In [ ]:
num_linking_snapshots = [2, 4, 6, 8, 12, 16, 24, 32]
for n_sp in num_linking_snapshots:
    with h5.File(fjlt_svd_file.format(n_sp), 'r') as f:
        fjlt_s = f["/s"][()]
    fig, axs = plt.subplots(ncols=1, nrows=2, sharex=True)
    plot_singular_values(axs, dir_s, fjlt_s)
    plt.savefig("../figures/svd_singular_values_n_sp_{}.pdf".format(n_sp),
                bbox_inches="tight", pad_inches=0.1,
                facecolor=None, edgecolor='auto'
                )

Illustration of relative error convergence with increasing number of snapshots.

In [ ]:
def plot_sv_error_convergence(ax, s, s_f, **kwargs):
    s, s_f = s[:sv_to_show+1], s_f[:sv_to_show+1]
    ax.plot(np.arange(len(s)), 100*np.abs(s-s_f)/s, marker="o", **kwargs)

    ax.set_ylabel(r"$\sigma_i$")
    ax.set_ylabel(r"Relative error in $\sigma$ $(\%)$")
    ax.set_xlabel(r"Mode index")
    ax.set_xlim([0, sv_to_show])

In [ ]:
colors = plt.cm.Blues(np.linspace(0.3,1,len(num_linking_snapshots)))

fig = plt.figure(figsize=(6.9/2, 2.1))
ax=fig.gca()
for i, n_sp in enumerate(num_linking_snapshots):
    with h5.File(fjlt_svd_file.format(n_sp), 'r') as f:
        fjlt_s = f["/s"][()]
        plot_sv_error_convergence(ax, dir_s, fjlt_s, color=colors[i], label=n_sp)

# ax.plot([0, len(dir_s)], [100*e, 100*e], color="k", alpha=0.5, ls="--")
ax.legend(ncols=4)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.savefig("../figures/svd_singular_value_errors.pdf",
                bbox_inches="tight", pad_inches=0.1,
                facecolor=None, edgecolor='auto'
                )

### Compare $V^H$ using PSD (Welch) and instantaneous frequency estimation (multiple signal classification algorithm (MUSIC))

In [ ]:
def normalise_v(v):
    for i in range(v.shape[1]):
        norm = 1 if sum(v[:, i])>=0 else -1
        v[:, i] *= norm
    return None

In [ ]:
with h5.File(dir_svd_file, 'r') as f:
    dir_v = np.matrix(f["/VH"][()]).H
normalise_v(dir_v)

In [ ]:
def get_v_error(V_a, V_b):
    cols_to_check = min(V_a.shape[1], V_b.shape[1])
    err = []
    for i in range(cols_to_check):
        v_a_col, v_b_col = V_a[:, i], V_b[:, i]
        diff = np.sum(np.abs(np.abs(v_a_col-v_b_col)))
        norm = np.sum(np.abs(v_a_col))
        err.append(100*diff/norm)
    return err

In [ ]:
def plot_v_psd_error(ax, dir_V, fjlt_V, n_sp, line_color):
    _, dir_v_psd = sig.welch(dir_V, axis=0)
    _, fjlt_v_psd = sig.welch(fjlt_V, axis=0)
    err = get_v_error(dir_v_psd, fjlt_v_psd)
    ax.plot(np.arange(len(err)), err, marker="o", color=line_color, label=n_sp)

In [ ]:
def instant_frequency_proc(f, dt):
    f = 2*np.pi*inst_freq(f, dt)
    return f[f==f]

In [ ]:
def plot_v_inst_freq_error(ax, dir_V, fjlt_V, n_sp, line_color):
    n_cols = sv_to_show+1
    temp = instant_frequency_proc(dir_V[:, 0], 1)
    f_inst_dir = np.zeros((len(temp), n_cols))
    f_inst_fjlt = np.zeros((len(temp), n_cols))

    for i in range(n_cols):
        f_inst_dir[:, i] = instant_frequency_proc(dir_V[:, i], 1)
        f_inst_fjlt[:, i] = instant_frequency_proc(fjlt_V[:, i], 1)

    err = get_v_error(f_inst_dir, f_inst_fjlt)
    ax.plot(np.arange(len(err)), err, marker="o", color=line_color, label=n_sp)

In [ ]:
# WELCH
plt.style.use("../../../mplstyles/paper_full.mplstyle")
fig, axs = plt.subplots(ncols=2, nrows=1, sharey=True, figsize=(6.9, 2.1))

for i, n_sp in enumerate(num_linking_snapshots):
    with h5.File(fjlt_svd_file.format(n_sp), 'r') as f:
        fjlt_v = np.matrix(f["/VH"][()]).H
        normalise_v(fjlt_v)
        plot_v_psd_error(axs[0], dir_v, fjlt_v, n_sp, line_color=colors[i])
        del fjlt_v
        gc.collect()

axs[0].set_xlim(0, sv_to_show)
axs[0].xaxis.set_major_locator(MaxNLocator(integer=True))
axs[0].set_ylim(0, 80)
# axs[0].legend(ncol=4)
axs[0].set_xlabel(r"Mode index")
axs[0].set_ylabel(r"Relative $V$ col PSD $l_1$ error (%)")



# INST. FREQ.
for i, n_sp in enumerate(num_linking_snapshots):
    with h5.File(fjlt_svd_file.format(n_sp), 'r') as f:
        fjlt_v = np.matrix(f["/VH"][()]).H
        normalise_v(fjlt_v)
        plot_v_inst_freq_error(axs[1], dir_v, fjlt_v, n_sp, line_color=colors[i])
        del fjlt_v
        gc.collect()

del dir_v
gc.collect()

axs[1].set_xlim(0, sv_to_show)
axs[1].set_ylim(0, 80)
axs[1].legend(ncol=4)
axs[1].set_xlabel(r"Mode index")
axs[1].xaxis.set_major_locator(MaxNLocator(integer=True))
axs[1].set_ylabel(r"Relative $V$ col inst. freq. $l_1$ error (%)")
plt.savefig("../figures/svd_v_error.pdf",
                bbox_inches="tight", pad_inches=0.1,
                facecolor=None, edgecolor='auto'
                )

### Reconstruction of $U$

Reconstruct $U$ using the direct file and load $U$ directly from the SVD.

In [ ]:
plt.style.use("../../../mplstyles/paper_full.mplstyle")

In [ ]:
def reconstruct_U(Q_file, V_file, num_s_vals):
    with h5.File(V_file, 'r') as f:
        V = (np.matrix(f["/VH"][:num_s_vals+1, :]).H)
        normalise_v(V)
        s = f["/s"][:num_s_vals+1]
        Vsinv = np.einsum("ij, j -> ij", V, 1/s)

    reader = TurboHDF5Reader(Q_file)
    Q = formQ(reader, snapshot_sequence)
    U = np.einsum("ij, jk -> ik", Q, Vsinv)
    return U

In [ ]:
def load_U(U_file):
    with h5.File(U_file, 'r') as f:
        U = f["/U"][()]
    return U

In [ ]:
num_snapshots = 6
dir_u = load_U(dir_svd_file)[:, :num_modes_to_visualise+1]
dir_u_reconst = reconstruct_U(Q_file, dir_svd_file, num_s_vals=num_modes_to_visualise)
fjlt_u_reconst = reconstruct_U(Q_file, fjlt_svd_file.format(num_snapshots), num_s_vals=num_modes_to_visualise)

In [ ]:
reader = TurboHDF5Reader(Q_file)
dir_u_field = reader.reconstruct_field(dir_u)
dir_u_reconst_field = reader.reconstruct_field(dir_u_reconst)
fjlt_u_reconst_field = reader.reconstruct_field(fjlt_u_reconst)

Reconstruct U from the FJLT data

In [ ]:
def visualise_field(ax, field, grid, vmin, vmax, h_pass, plot_offset):
    for i in range(3):
        for j in [-1, 0, 1]:
            ax.pcolormesh(grid[i][:, :, 0], grid[i][:, :, 1]+j*h_pass-plot_offset,
                        field[i],
                        vmin=vmin, vmax=vmax, shading="gouraud", cmap="RdBu_r", rasterized=True)
    return None

In [ ]:
def compare_snapshots(axs, field_A, field_B, turboreader, visualisation_variable, snapshot_index, plot_offset):
    field_A = [field_A[snapshot_index][i][:, :, visualisation_variable] for i in range(turboreader.num_regions)]
    field_B = [field_B[snapshot_index][i][:, :, visualisation_variable] for i in range(turboreader.num_regions)]

    # Multiply by ±1 to get the same field
    A_factor = 1 if field_A[0][0, 0] >= 0 else -1
    B_factor = 1 if field_B[0][0, 0] >= 0 else -1
    field_A = [A_factor*field_A[i] for i in range(turboreader.num_regions)]
    field_B = [B_factor*field_B[i] for i in range(turboreader.num_regions)]

    # vminA = np.min([np.min(region) for region in field_A])
    # vmaxA = np.max([np.max(region) for region in field_A])

    vminA = np.min(np.min(field_A[0]))
    vmaxA = np.max(np.max(field_A[0]))

    vminB = np.min(np.min(field_B[0]))
    vmaxB = np.max(np.max(field_B[0]))

    vmin = min([vminA, vminB])
    vmax = max([vmaxA, vmaxB])
    vabs = max([abs(vmin), abs(vmax)])


    grid = turboreader.load_grid()

    visualise_field(axs[0], field_A, grid, -vabs, vabs, 0.6, plot_offset)
    visualise_field(axs[1], field_B, grid, -vabs, vabs, 0.6, plot_offset)

    for ax in axs:
        ax.set_aspect("equal")

    for i in range(2):
        axs[i].set_xlabel(r"$x$")
    axs[0].set_ylabel(r"$y$")

    axs[0].set_title("Direct")
    axs[1].set_title("FJLT reconstructed")

    for ax in axs:
        ax.axis("off")

    return None

### Show the mode comparison
Currently plotting just the field with normalisation in the upstream region in both cases. 
Reconstruction from the direct and FJLT data.

In [ ]:
plt.style.use("../../../mplstyles/paper_full_75pc.mplstyle")
plot_offset = 0.6*3 + 0.3

fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(5.18, 2*8), sharey=True)

for mode in range(6):
    compare_snapshots(axs, dir_u_field, fjlt_u_reconst_field, reader, 0, mode, plot_offset*mode)
plt.savefig("../figures/svd_u_reconst_pod_modes_num_snaps_{}.pdf".format(num_snapshots),
            bbox_inches="tight", pad_inches=0.1,
            facecolor=None, edgecolor='auto', dpi=300
            )
# del reader
# gc.collect()